<a href="https://colab.research.google.com/github/bishair/Pirna/blob/main/ModelPirna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and upload excel files for groundwater level and river water level separately

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
uploaded_file = files.upload()

Saving riverL.xlsx to riverL (4).xlsx


# Main Functions
**merge_and_resample:**

Merge and preprocess data from two Excel files containing groundwater and river water level data along with their respective dates, function  reads the two files, merges the data on the date column, resamples it to an hourly frequency, and then applies forward fill to handle missing hours.

**preprocess_data_for_lstm:**

Handle missing values for groundwater level and river water level by using the previous value. Normalize the data in the range 0 to 1. Return numpy arrays for X and y and create sequences using windows of given length.

**Train and save model:**
Specify optimizer, loss function, batch size, epochs and number of neurons and layers for model training

In [ ]:
#Use consistent time data, in case of missing observations use forward fill to have consistent time data
#date_format= '%Y-%m-%d %H:%M:%S'

def merge_and_resample(file_groundwater, file_river, date_format_groundwater, date_format_river):
    # Read the groundwater level data
    gw_data = pd.read_excel(file_groundwater)
    gw_data['Date'] = pd.to_datetime(gw_data['Date'], format=date_format_groundwater)

    # Read the river water level data
    river_data = pd.read_excel(file_river)
    river_data['Date'] = pd.to_datetime(river_data['Date'], format=date_format_river)

    # Merge the two datasets on the 'Date' column
    merged_data = pd.merge_asof(gw_data.sort_values('Date'), river_data.sort_values('Date'), on='Date', direction='nearest')

    # Set 'Date' as the index
    merged_data.set_index('Date', inplace=True)

    # Create a new date range that starts from the first timestamp in merged_data
    start_date = merged_data.index.min()
    end_date = merged_data.index.max()
    new_date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    # Reindex and resample merged_data with the new date range
    resampled_data = merged_data.reindex(new_date_range).ffill().bfill()

    return resampled_data



In [ ]:

def preprocess_data_for_lstm(data, n_steps):
    """
    Preprocess the data for LSTM model to predict 'G10' using past values of 'G10' and 'River'.
    - data: pandas DataFrame containing 'G10' and 'River' columns.
    - n_steps: number of time steps to use for predicting the next time step.

    Returns:
    - X, y: numpy arrays of features and labels suitable for LSTM model.
    """
    # Handling missing values using forward fill
    data = data.fillna(method='ffill')

    # Normalizing the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)

    # Creating sequences
    X, y = [], []
    for i in range(n_steps, len(data_scaled)):
        X.append(data_scaled[i-n_steps:i, :])
        y.append(data_scaled[i, 0])

    X, y = np.array(X), np.array(y)
    # Split the data into training and testing sets
    train_size = int(len(X) * 0.7)
    test_size = len(X) - train_size
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return X_train, y_train, X_test, y_test



In [ ]:
def train_model(x_train, y_train, input_shape, epochs=100, batch_size=32):
    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=input_shape),
        LSTM(units=50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
    return model

In [ ]:
def save_model(model, filename='my_lstm_model.h5'):
    model.save(filename)
    return model

In [ ]:
def evaluate_model(model, x_test, y_test):
    # Evaluate the model's performance on the test data
    # It returns a list: [loss, mse]
    loss = model.evaluate(x_test, y_test)
    return loss


# Forward fill the missing hours by calling merge_and_resample function

In [ ]:
merged_df = merge_and_resample('groundwater.xlsx','riverL.xlsx', '%d/%m/%Y %H:%M' ,'%d/%m/%Y %H:%M')

# Preprocess and define window size for LSTM model by using function(preprocess_data_for_lstm)

In [ ]:
n_steps = 24
X_train, y_train, X_test, y_test = preprocess_data_for_lstm(merged_df, n_steps)

# Train and save model

In [ ]:
"""
input_shape  includes two dimensions: (time_steps, features)
Time Steps: Number of time intervals or sequences each input sample consists of.
Features: Number of variables observed at each time step.
"""

input_shape=(X_train.shape[1], X_train.shape[2])
epochs=40
batch_size=64
trained_model = train_model(X_train, y_train, input_shape, epochs, batch_size)

Epoch 1/40
192/192 [==============================] - 6s 17ms/step - loss: 7.2515e-04
Epoch 2/40
192/192 [==============================] - 3s 18ms/step - loss: 7.7589e-06
Epoch 3/40
192/192 [==============================] - 4s 19ms/step - loss: 8.0858e-06
Epoch 4/40
192/192 [==============================] - 3s 17ms/step - loss: 9.7840e-06
Epoch 5/40
192/192 [==============================] - 3s 17ms/step - loss: 7.1974e-06
Epoch 6/40
192/192 [==============================] - 4s 20ms/step - loss: 1.1938e-05
Epoch 7/40
192/192 [==============================] - 3s 17ms/step - loss: 7.4592e-06
Epoch 8/40
192/192 [==============================] - 3s 17ms/step - loss: 7.2645e-06
Epoch 9/40
192/192 [==============================] - 4s 20ms/step - loss: 6.3398e-06
Epoch 10/40
192/192 [==============================] - 3s 17ms/step - loss: 6.7521e-06
Epoch 11/40
192/192 [==============================] - 3s 17ms/step - loss: 8.6211e-06
Epoch 12/40
192/192 [==============================]

In [ ]:
saved_model = save_model(trained_model)
print(saved_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Evaluate Model by calculating MSE

In [ ]:
evaluate_model(saved_model, X_test, y_test)


165/165 [==============================] - 1s 5ms/step - loss: 4.3812e-09


4.381248253082504e-09

# ------------------------**Anomaly detection in new data**------------------------------

In [ ]:
uploaded_file = files.upload()

Saving riverL1.xlsx to riverL1.xlsx
